<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/3%20Aprendizaje%20profundo%20(II)/Sesion%205/B3_GRU_AEP_PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![IDAL](https://i.imgur.com/tIKXIG1.jpg)  

#**Máster en Inteligencia Artificial Avanzada y Aplicada:  IA^3**
---

#<strong><center>Predicción de consumo energético: GRU vs LSTM</center></strong>

En este cuaderno, utilizaremos un modelo GRU para una tarea de predicción de series temporales y compararemos el rendimiento del modelo GRU contra un modelo LSTM. Los modelos LSTM los vais a estudiar en detalle en próximas sesiones, pero aquí va un pequeño adelanto comparativo. 

El conjunto de datos que utilizaremos es el conjunto de datos de consumo de energía por hora que se puede encontrar en [Kaggle](https://www.kaggle.com/robikscube/hourly-energy-consumption). El conjunto de datos contiene datos de consumo de energía en diferentes regiones de los Estados Unidos registrados cada hora.

El objetivo de esta implementación es crear un modelo que pueda predecir con exactitud el uso de energía en la siguiente hora dados los datos históricos de uso. Utilizaremos tanto el modelo GRU como el LSTM para entrenar en un conjunto de datos históricos y evaluaremos ambos modelos en un conjunto de pruebas no visto. Para ello, comenzaremos con la selección de características, el preprocesamiento de datos, seguido de la definición, el entrenamiento y, finalmente, la evaluación de los modelos.

Utilizaremos la librería PyTorch para implementar ambos tipos de modelos junto con otras librerías de Python habituales en el análisis de datos.

In [ ]:
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler




In [ ]:
data_dir = '/content/AEP_data/'
pd.read_csv(data_dir + 'AEP_hourly.csv').head()

,Datetime,AEP_MW
0,2004-12-31 01:00:00,13478.0
1,2004-12-31 02:00:00,12865.0
2,2004-12-31 03:00:00,12577.0
3,2004-12-31 04:00:00,12517.0
4,2004-12-31 05:00:00,12670.0


Tenemos un total de **12** archivos *.csv* que contienen datos de tendencias energéticas horarias (*'est_hourly.paruqet'* y *'pjm_hourly_est.csv'* no se utilizan). En nuestro siguiente paso, leeremos estos archivos y preprocesaremos estos datos en este orden:
- Obteniendo los datos de tiempo de cada paso de tiempo individual y generalizándolos
    - Hora del día *es decir, 0-23*.
    - Día de la semana *es decir, del 1 al 7*.
    - Mes *es 1-12*.
    - Día del año *es decir, 1-365*.
    
    
- Escala los datos a valores entre 0 y 1
    - Los algoritmos tienden a funcionar mejor o a converger más rápidamente cuando las características están en una escala relativamente similar y/o se acercan a una distribución normal
    - La escala preserva la forma de la distribución original y no reduce la importancia de los valores atípicos.
    
    
- Agrupar los datos en secuencias que se utilizarán como entradas del modelo y almacenar sus correspondientes etiquetas
    - La **longitud de la secuencia** o **período de espera** es el número de puntos de datos de la historia que el modelo utilizará para hacer la predicción
    - La etiqueta será el siguiente punto de datos en el tiempo después del último de la secuencia de entrada
    

- Las entradas y las etiquetas se dividirán en conjuntos de entrenamiento y de prueba.

In [ ]:
# The scaler objects will be stored in this dictionary so that our output test data from the model can be re-scaled during evaluation
label_scalers = {}

train_x = []
test_x = {}
test_y = {}

for file in tqdm_notebook(os.listdir(data_dir)): 
    # Skipping the files we're not using
    if file[-4:] != ".csv" or file == "pjm_hourly_est.csv":
        continue
        
    print(file)
    # Store csv file in a Pandas DataFrame
    df = pd.read_csv(data_dir + file, parse_dates=[0])
    # Processing the time data into suitable input formats
    df['hour'] = df.apply(lambda x: x['Datetime'].hour,axis=1)
    df['dayofweek'] = df.apply(lambda x: x['Datetime'].dayofweek,axis=1)
    df['month'] = df.apply(lambda x: x['Datetime'].month,axis=1)
    df['dayofyear'] = df.apply(lambda x: x['Datetime'].dayofyear,axis=1)
    df = df.sort_values("Datetime").drop("Datetime",axis=1)
    
    # Scaling the input data
    sc = MinMaxScaler()
    label_sc = MinMaxScaler()
    data = sc.fit_transform(df.values)
    # Obtaining the Scale for the labels(usage data) so that output can be re-scaled to actual value during evaluation
    label_sc.fit(df.iloc[:,0].values.reshape(-1,1))
    label_scalers[file] = label_sc
    
    # Define lookback period and split inputs/labels
    lookback = 90
    inputs = np.zeros((len(data)-lookback,lookback,df.shape[1]))
    labels = np.zeros(len(data)-lookback)
    
    for i in range(lookback, len(data)):
        inputs[i-lookback] = data[i-lookback:i]
        labels[i-lookback] = data[i,0]
    inputs = inputs.reshape(-1,lookback,df.shape[1])
    labels = labels.reshape(-1,1)
    
    # Split data into train/test portions and combining all data from different files into a single array
    test_portion = int(0.1*len(inputs))
    if len(train_x) == 0:
        train_x = inputs[:-test_portion]
        train_y = labels[:-test_portion]
    else:
        train_x = np.concatenate((train_x,inputs[:-test_portion]))
        train_y = np.concatenate((train_y,labels[:-test_portion]))
    test_x[file] = (inputs[-test_portion:])
    test_y[file] = (labels[-test_portion:])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


PJME_hourly.csv
DAYTON_hourly.csv
DOM_hourly.csv
DUQ_hourly.csv
PJMW_hourly.csv
EKPC_hourly.csv
COMED_hourly.csv
AEP_hourly.csv
FE_hourly.csv
DEOK_hourly.csv
NI_hourly.csv
PJM_Load_hourly.csv



In [ ]:
print(train_x.shape)

(980185, 90, 5)


In [ ]:
train_x[1:5]

array([[[0.31014432, 0.08695652, 0.16666667, 0.        , 0.        ],
        [0.29101443, 0.13043478, 0.16666667, 0.        , 0.        ],
        [0.28136522, 0.17391304, 0.16666667, 0.        , 0.        ],
        ...,
        [0.4217634 , 0.73913043, 0.66666667, 0.        , 0.00821918],
        [0.48806489, 0.7826087 , 0.66666667, 0.        , 0.00821918],
        [0.49971558, 0.82608696, 0.66666667, 0.        , 0.00821918]],

       [[0.29101443, 0.13043478, 0.16666667, 0.        , 0.        ],
        [0.28136522, 0.17391304, 0.16666667, 0.        , 0.        ],
        [0.28469399, 0.2173913 , 0.16666667, 0.        , 0.        ],
        ...,
        [0.48806489, 0.7826087 , 0.66666667, 0.        , 0.00821918],
        [0.49971558, 0.82608696, 0.66666667, 0.        , 0.00821918],
        [0.48446224, 0.86956522, 0.66666667, 0.        , 0.00821918]],

       [[0.28136522, 0.17391304, 0.16666667, 0.        , 0.        ],
        [0.28469399, 0.2173913 , 0.16666667, 0.        , 0. 

Tenemos un total de 980.185 secuencias de datos de entrenamiento

Para mejorar la velocidad de nuestro entrenamiento, podemos procesar los datos en lotes para que el modelo no necesite actualizar sus pesos con tanta frecuencia. Las clases *Dataset* y *DataLoader* de Torch son útiles para dividir nuestros datos en lotes y mezclarlos.

In [ ]:
batch_size = 1024

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

También podemos comprobar si tenemos alguna GPU para acelerar nuestro tiempo de entrenamiento. Si utilizas GPU para ejecutar este código, el tiempo de entrenamiento se reducirá considerablemente.

In [ ]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
device

device(type='cuda')

A continuación, definiremos la estructura de los modelos GRU y LSTM. Ambos modelos tienen la misma estructura, con la única diferencia de la **capa recurrente** (GRU/LSTM) y la inicialización del estado oculto. El estado oculto para el LSTM es una tupla que contiene tanto el **estado de las celdas** como el **estado oculto**, mientras que el GRU sólo tiene un único estado oculto.

In [ ]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

El proceso de entrenamiento lo vamos a definir en una función a continuación para que podamos reproducirlo para ambos modelos, especificandolo como parámetro. 

Ambos modelos tendrán el mismo número de **dimensiones** en el *estado oculto* y en las *capas*, se entrenarán con el mismo número de **epochs** y la misma **tasa de aprendizaje**, y se entrenarán y probarán con el mismo conjunto de datos.

Con el fin de comparar el rendimiento de ambos modelos, también haremos un seguimiento del tiempo que tarda el modelo en entrenarse y, finalmente, compararemos la precisión final de ambos modelos en el conjunto de pruebas. Para medir la precisión, utilizaremos el *Porcentaje Medio Absoluto de Error Simétrico (sMAPE)* para evaluar los modelos. El *sMAPE* es la suma de la **diferencia absoluta** entre los valores predichos y los reales dividida por la media de los valores predichos y los reales, lo que da un porcentaje que mide la cantidad de error. 

Esta es la fórmula de *sMAPE*:

$sMAPE = \frac{100%}{n}\sum_{t=1}^n \frac{|F_t - A_t|}{(|F_t + A_t|)/2}$


In [ ]:
def train(train_loader, learn_rate, hidden_dim=256, EPOCHS=5, model_type="GRU"):
    
    # Setting common hyperparameters
    input_dim = next(iter(train_loader))[0].shape[2]
    output_dim = 1
    n_layers = 2
    # Instantiating the models
    if model_type == "GRU":
        model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
    else:
        model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
    model.to(device)
    
    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of {} model".format(model_type))
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        start_time = time.clock()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            if model_type == "GRU":
                h = h.data
            else:
                h = tuple([e.data for e in h])
            model.zero_grad()
            
            out, h = model(x.to(device).float(), h)
            loss = criterion(out, label.to(device).float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.clock()
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, EPOCHS, avg_loss/len(train_loader)))
        print("Time Elapsed for Epoch: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model

def evaluate(model, test_x, test_y, label_scalers):
    model.eval()
    outputs = []
    targets = []
    start_time = time.clock()
    for i in test_x.keys():
        inp = torch.from_numpy(np.array(test_x[i]))
        labs = torch.from_numpy(np.array(test_y[i]))
        h = model.init_hidden(inp.shape[0])
        out, h = model(inp.to(device).float(), h)
        outputs.append(label_scalers[i].inverse_transform(out.cpu().detach().numpy()).reshape(-1))
        targets.append(label_scalers[i].inverse_transform(labs.numpy()).reshape(-1))
    print("Evaluation Time: {}".format(str(time.clock()-start_time)))
    sMAPE = 0
    for i in range(len(outputs)):
        sMAPE += np.mean(abs(outputs[i]-targets[i])/(targets[i]+outputs[i])/2)/len(outputs)
    print("sMAPE: {}%".format(sMAPE*100))
    return outputs, targets, sMAPE

In [ ]:
lr = 0.001
gru_model = train(train_loader, lr, model_type="GRU")

Starting Training of GRU model
Epoch 1......Step: 200/957....... Average Loss for Epoch: 0.005936735754075926
Epoch 1......Step: 400/957....... Average Loss for Epoch: 0.003330760081371409
Epoch 1......Step: 600/957....... Average Loss for Epoch: 0.0023780939747909237
Epoch 1......Step: 800/957....... Average Loss for Epoch: 0.0018692187203487266
Epoch 1/5 Done, Total Loss: 0.0016085070442723873
Time Elapsed for Epoch: 125.908102 seconds
Epoch 2......Step: 200/957....... Average Loss for Epoch: 0.0002401996850676369
Epoch 2......Step: 400/957....... Average Loss for Epoch: 0.00023332162822043757
Epoch 2......Step: 600/957....... Average Loss for Epoch: 0.00022223237501748372
Epoch 2......Step: 800/957....... Average Loss for Epoch: 0.0002121916153919301
Epoch 2/5 Done, Total Loss: 0.00020969844344894371
Time Elapsed for Epoch: 134.599167 seconds
Epoch 3......Step: 200/957....... Average Loss for Epoch: 0.0001650260577298468
Epoch 3......Step: 400/957....... Average Loss for Epoch: 0.00

In [ ]:
lstm_model = train(train_loader, lr, model_type="LSTM")

Starting Training of LSTM model
Epoch 1......Step: 200/957....... Average Loss for Epoch: 0.009550642013200559
Epoch 1......Step: 400/957....... Average Loss for Epoch: 0.005338132382748881
Epoch 1......Step: 600/957....... Average Loss for Epoch: 0.003776723096477023
Epoch 1......Step: 800/957....... Average Loss for Epoch: 0.002958885270782048
Epoch 1/5 Done, Total Loss: 0.002542278045545602
Time Elapsed for Epoch: 182.40060799999992 seconds
Epoch 2......Step: 200/957....... Average Loss for Epoch: 0.0003535627499513794
Epoch 2......Step: 400/957....... Average Loss for Epoch: 0.0003139567621474271
Epoch 2......Step: 600/957....... Average Loss for Epoch: 0.00029095206363611697
Epoch 2......Step: 800/957....... Average Loss for Epoch: 0.000271598013532639
Epoch 2/5 Done, Total Loss: 0.0002602678715574235
Time Elapsed for Epoch: 183.7056389999999 seconds
Epoch 3......Step: 200/957....... Average Loss for Epoch: 0.0001842213617055677
Epoch 3......Step: 400/957....... Average Loss for E

Como podemos ver en el tiempo de entrenamiento de ambos modelos, el modelo GRU es el claro ganador en términos de **velocidad**, como hemos mencionado anteriormente. Esto es algo lógico ya que la arquitectura GTU requiere **menos parametros de entrenamiento**.

Pasando a medir la precisión de ambos modelos, ahora utilizaremos nuestra función evaluate() y el conjunto de datos de prueba.

In [ ]:
gru_outputs, targets, gru_sMAPE = evaluate(gru_model, test_x, test_y, label_scalers)

Evaluation Time: 4.140718000000106
sMAPE: 0.33043812440727127%


In [ ]:
gru_outputs[1:5]

[array([2090.7698, 2077.991 , 2105.4268, ..., 2410.7932, 2263.2756,
        2073.1133], dtype=float32),
 array([ 9827.664,  9934.596, 10016.663, ..., 13441.883, 12503.295,
        11408.929], dtype=float32),
 array([1262.1996, 1308.6973, 1348.8604, ..., 1896.7057, 1800.5834,
        1663.644 ], dtype=float32),
 array([6637.2   , 7046.5054, 7352.679 , ..., 6330.481 , 5971.9004,
        5453.276 ], dtype=float32)]

In [ ]:
targets[1:5]

[array([2063., 2061., 2119., ..., 2405., 2250., 2042.]),
 array([ 9618.,  9803.,  9870., ..., 13312., 12390., 11385.]),
 array([1247., 1298., 1352., ..., 1901., 1789., 1656.]),
 array([6700., 7248., 7319., ..., 6325., 5892., 5489.])]

In [ ]:
df.tail()

,PJM_Load_MW,hour,dayofweek,month,dayofyear
24157,36392.0,20,0,12,365
24158,35082.0,21,0,12,365
24159,33890.0,22,0,12,365
24160,32590.0,23,0,12,365
24161,31569.0,0,1,1,1


In [ ]:
lstm_outputs, targets, lstm_sMAPE = evaluate(lstm_model, test_x, test_y, label_scalers)

Evaluation Time: 5.306215000000066
sMAPE: 0.26701289796413774%


Aunque el modelo LSTM puede haber cometido menos errores y haber superado ligeramente al modelo GRU en términos de precisión del rendimiento, la diferencia es insignificante y, por tanto, no concluyente. 

Se han realizado numerosas  pruebas en las que se comparan estos dos modelos, pero en general no ha habido un ganador claro en cuanto a cuál es la mejor arquitectura en general. La realidad indica que la mejor solución a cada problema suele ser muy específica y dependiente de la naturaleza del problea, tipo de datos, etc. sin que haya una regla aplicable para todos los casos.

## Referencias

* *texto en cursiva* Documento inspirado en los ejemplos disponibles en FloidHub: https://github.com/floydhub/examples
* Doc oficial Pytorch https://pytorch.org/docs/stable/generated/torch.nn.RNN.html
* https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html

## Fin del cuaderno